Grid Search：网格搜索，一种调参手段；穷举搜索：在所有候选的参数选择中，通过循环遍历，尝试每一种可能性，表现最好的参数就是最终的结果。其原理就像是在数组里找最大值。

# 网格搜索

sklearn.model_selection.GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)

- estimator: 模型
- param_grid: 要循环的参数，字典或含字典的列表
- scoring: 评估指标
- cv: 交叉验证折数

属性：

- best_estimator: 最优模型
- best_score_: 最高分数
- best_params_: 最优参数

## 未调参

In [1]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC

iris = load_iris()
svc = SVC()
svc.fit(iris.data, iris.target)
svc.score(iris.data, iris.target)

0.9733333333333334

## 调参后

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

iris = load_iris()

parameters = {'gamma': [0.001,0.01,0.1,1,10,100],
              'C': [0.001,0.01,0.1,1,10,100]}

svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)

# 最高分数
print(clf.best_score_)
# 最优模型
print(clf.best_estimator_)
# 最优参数
print(clf.best_params_)

0.9800000000000001
SVC(C=1, gamma=0.1)
{'C': 1, 'gamma': 0.1}


# 随机搜索

[参考](https://www.cnblogs.com/wj-1314/p/10422159.html)

我们在搜索超参数的时候，如果超参数个数较少（三四个或者更少），那么我们可以采用网格搜索，一种穷尽式的搜索方法。但是当超参数个数比较多的时候，我们仍然采用网格搜索，那么搜索所需时间将会指数级上升。

所以有人就提出了随机搜索的方法，随机在超参数空间中搜索几十几百个点，其中就有可能有比较小的值。这种做法比上面稀疏化网格的做法快，而且实验证明，随机搜索法结果比稀疏网格法稍好。

sklearn.model_selection.RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score=nan, return_train_score=False)

- estimator: 模型
- param_distributions: 要循环的参数，字典或含字典的列表
- n_iter: 迭代次数
- scoring: 评估指标
- cv: 交叉验证折数
- random_state: 随机数

属性：

- best_estimator: 最优模型
- best_score_: 最高分数
- best_params_: 最优参数

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
import numpy as np

iris = load_iris()

parameters = {'gamma': np.arange(0, 100, 0.1),
              'C': [0.001,0.01,0.1,1,10,100]}

svc = SVC()

clf = RandomizedSearchCV(svc, parameters, n_iter=300)

clf.fit(iris.data, iris.target)

print(clf.best_score_)
print(clf.best_params_)

0.9666666666666668
{'gamma': 4.1000000000000005, 'C': 1}
